# Prepare input data for RF feature selection

In [20]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn import preprocessing

## Set Paths

In [21]:
ROOT = Path('/gpfs/work1/0/wtrcycle/users/jaerts/model_refinement_pub/')
ATTRDIR = f'{ROOT}/catchment_attributes/'
RESULTS = f'{ROOT}/results/new/'

## Create single dataframe with CAMELS-GB classifiers and Hydrological Signatures
1. Load individual catchment classification files
2. Drop redundant variables
3. Append dataframe to list
4. Add McMillan hydrological signatures

In [22]:
dataframes = []

In [23]:
# Topo class
df = pd.read_csv(f"{ATTRDIR}/camels-gb_attributes/CAMELS_GB_topographic_attributes.csv", index_col='gauge_id')
df = df.drop(columns=['gauge_name', 'gauge_lat', 'gauge_lon', 'gauge_easting','gauge_northing', 'elev_min', 'elev_10', 'elev_50', 'elev_90', 'elev_max'])

dataframes.append(df)

In [24]:
# Climate class
df = pd.read_csv(f"{ATTRDIR}/camels-gb_attributes/CAMELS_GB_climatic_attributes.csv", index_col='gauge_id')
df = df.fillna(0)

# Hot encode high_prec_timing and low_prec_timing
list_high_prec = []
list_low_prec = []

for index, row in df.iterrows():
    
    # High prec timing
    if row.high_prec_timing == 0:
        list_high_prec.append(0)
    if row.high_prec_timing == 'son':
        list_high_prec.append(1)
    if row.high_prec_timing == 'djf':
        list_high_prec.append(2)
    if row.high_prec_timing == 'jja':    
        list_high_prec.append(3)
        
    # Low prec timing
    if row.low_prec_timing == 0:
        list_low_prec.append(0)
    if row.low_prec_timing == 'jja':
        list_low_prec.append(1)
    if row.low_prec_timing == 'mam':
        list_low_prec.append(2)
        
# Overwrite existing columns
df.high_prec_timing = list_high_prec
df.low_prec_timing = list_low_prec    
    
dataframes.append(df)
df

,p_mean,pet_mean,aridity,p_seasonality,frac_snow,high_prec_freq,high_prec_dur,high_prec_timing,low_prec_freq,low_prec_dur,low_prec_timing
gauge_id,,,,,,,,,,,
10002,2.29,1.26,0.55,-0.03,0.03,13.96,1.16,1,207.89,3.17,1
10003,2.31,1.25,0.54,-0.01,0.03,13.24,1.17,1,209.89,3.20,1
1001,2.65,1.15,0.44,-0.15,0.03,10.98,1.11,1,179.87,2.93,1
101002,2.31,1.45,0.63,-0.23,0.01,20.60,1.17,1,245.03,4.46,1
101005,2.29,1.45,0.63,-0.23,0.01,20.47,1.19,1,244.80,4.47,1
...,...,...,...,...,...,...,...,...,...,...,...
95001,6.20,1.05,0.17,-0.25,0.03,8.16,1.14,1,125.29,2.69,1
96001,3.06,1.10,0.36,-0.19,0.04,9.73,1.11,1,168.49,2.89,1
96002,3.81,1.08,0.28,-0.28,0.03,8.53,1.11,1,149.14,2.87,1


In [25]:
# Humaninfluence class
## Contains many NaN values > 20%, drop...
df = pd.read_csv(f"{ATTRDIR}/camels-gb_attributes/CAMELS_GB_humaninfluence_attributes.csv", usecols=['gauge_id','surfacewater_abs','groundwater_abs']
               )   
df = df.set_index('gauge_id')
# dataframes.append(df)
# df

In [26]:
# Hydrogeology class
df = pd.read_csv(f"{ATTRDIR}/camels-gb_attributes/CAMELS_GB_hydrogeology_attributes.csv", index_col='gauge_id')
# df = df.drop(columns=['low_nsig_perc','nsig_low_perc'])
df = df.drop(columns=['inter_high_perc', 'inter_low_perc', 'frac_high_perc', 'frac_mod_perc','frac_low_perc','low_nsig_perc','nsig_low_perc'])
dataframes.append(df)

print(len(df.dropna()))
df.fillna(0).astype(bool).sum(axis=0)
df

671


,inter_mod_perc,no_gw_perc
gauge_id,,
10002,18.53,57.67
10003,13.14,63.72
1001,10.20,87.55
101002,30.76,19.42
101005,44.36,17.96
...,...,...
95001,13.32,26.00
96001,9.51,89.75
96002,10.37,80.58


In [27]:
# Hydrology class
df = pd.read_table(f"{ATTRDIR}/hydrological_signatures/CAMELS_GB_signatures.txt", sep=',')
df['gauge_id'] = pd.read_csv(f"{ATTRDIR}/camels-gb_attributes/CAMELS_GB_climatic_attributes.csv", index_col='gauge_id').index.to_list() #add basin id
df = df.set_index('gauge_id')



df = df.drop(columns=['Q_mean_error_str', 'TotalRR_error_str','QP_elasticity_error_str', 'FDC_slope_error_str','BFI_error_str', 'HFD_mean_error_str',
                      'Q5_error_str', 'Q95_error_str','high_Q_freq_error_str', 'high_Q_dur_error_str','low_Q_freq_error_str','low_Q_dur_error_str',
                      'low_Q_dur_error_str','zero_Q_freq_error_str','zero_Q_freq','TotalRR'])
dataframes.append(df)

print(len(df.dropna()))
df

664


,Q_mean,QP_elasticity,FDC_slope,BFI,HFD_mean,Q5,Q95,high_Q_freq,high_Q_dur,low_Q_freq,low_Q_dur
gauge_id,,,,,,,,,,,
10002,1.477024,1.866296,-1.807712,0.669621,121.000000,0.40,3.88,0.007303,1.230769,0.000000,0.000000
10003,1.410155,0.514065,-1.750876,0.716257,127.333333,0.39,3.66,0.003651,1.333333,0.000000,0.000000
1001,1.686225,1.082786,-3.672701,0.492377,108.666667,0.10,5.49,0.018713,1.640000,0.248745,9.909091
101002,1.170114,2.033642,-1.939588,0.683762,137.666667,0.38,3.34,0.007759,1.133333,0.000000,0.000000
101005,0.963318,1.469087,-1.948980,0.591707,132.833333,0.28,3.03,0.023733,1.529412,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
95001,5.098955,0.837431,-1.638759,0.702641,136.833333,1.43,11.29,0.000000,0.000000,0.015974,11.666667
96001,2.255367,0.851758,-3.507578,0.429836,114.333333,0.18,7.96,0.025103,1.341463,0.238704,9.867925
96002,2.979311,0.258927,-2.815606,0.521215,117.333333,0.30,9.05,0.005477,1.090909,0.167503,12.233333


In [28]:
df

,Q_mean,QP_elasticity,FDC_slope,BFI,HFD_mean,Q5,Q95,high_Q_freq,high_Q_dur,low_Q_freq,low_Q_dur
gauge_id,,,,,,,,,,,
10002,1.477024,1.866296,-1.807712,0.669621,121.000000,0.40,3.88,0.007303,1.230769,0.000000,0.000000
10003,1.410155,0.514065,-1.750876,0.716257,127.333333,0.39,3.66,0.003651,1.333333,0.000000,0.000000
1001,1.686225,1.082786,-3.672701,0.492377,108.666667,0.10,5.49,0.018713,1.640000,0.248745,9.909091
101002,1.170114,2.033642,-1.939588,0.683762,137.666667,0.38,3.34,0.007759,1.133333,0.000000,0.000000
101005,0.963318,1.469087,-1.948980,0.591707,132.833333,0.28,3.03,0.023733,1.529412,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
95001,5.098955,0.837431,-1.638759,0.702641,136.833333,1.43,11.29,0.000000,0.000000,0.015974,11.666667
96001,2.255367,0.851758,-3.507578,0.429836,114.333333,0.18,7.96,0.025103,1.341463,0.238704,9.867925
96002,2.979311,0.258927,-2.815606,0.521215,117.333333,0.30,9.05,0.005477,1.090909,0.167503,12.233333


In [29]:
# Hydrometry class
## Contains many NaN values, drop for now...
df = pd.read_csv(f"{ATTRDIR}/camels-gb_attributes/CAMELS_GB_hydrometry_attributes.csv", index_col='gauge_id')
df.fillna(0).astype(bool).sum(axis=0)

station_type          670
flow_period_start     671
flow_period_end       671
flow_perc_complete    671
bankfull_flow         361
structurefull_flow    263
q5_uncert_upper       436
q5_uncert_lower       435
q25_uncert_upper      498
q25_uncert_lower      497
q50_uncert_upper      503
q50_uncert_lower      502
q75_uncert_upper      501
q75_uncert_lower      500
q95_uncert_upper      476
q95_uncert_lower      475
q99_uncert_upper      421
q99_uncert_lower      420
quncert_meta          671
dtype: int64

In [30]:
# Landcover class
df = pd.read_csv(f"{ATTRDIR}/camels-gb_attributes/CAMELS_GB_landcover_attributes.csv", index_col='gauge_id')
# df = df.drop(columns=['dom_land_cover'])

# Hot encode dom_land_cover
list_dom_land_cover = []

for index, row in df.iterrows():
    
    # High prec timing
    if row.dom_land_cover == 'Grass and Pasture':
        list_dom_land_cover.append(1)
    if row.dom_land_cover == 'Crops':
        list_dom_land_cover.append(2)
    if row.dom_land_cover == 'Shrubs':
        list_dom_land_cover.append(3)
    if row.dom_land_cover == 'Urban':    
        list_dom_land_cover.append(4)
    if row.dom_land_cover == 'Evergreen Woodland':
        list_dom_land_cover.append(5)
    if row.dom_land_cover == 'Deciduous Woodland':    
        list_dom_land_cover.append(6)        

df.dom_land_cover = list_dom_land_cover
dataframes.append(df)

print(len(df.dropna()))
df


671


,dwood_perc,ewood_perc,grass_perc,shrub_perc,crop_perc,urban_perc,inwater_perc,bares_perc,dom_land_cover
gauge_id,,,,,,,,,
10002,3.89,5.41,42.48,6.16,40.37,1.72,0.13,0.01,1
10003,4.74,3.06,36.88,0.31,54.06,1.04,0.06,0.03,2
1001,0.41,12.37,39.84,41.89,1.78,0.21,3.36,0.30,3
101002,6.20,0.30,42.78,0.38,46.34,3.28,0.49,0.56,2
101005,4.60,0.12,59.24,0.25,31.08,4.78,0.05,0.00,1
...,...,...,...,...,...,...,...,...,...
95001,0.36,0.14,7.92,79.81,0.01,0.01,7.78,3.99,3
96001,0.05,24.48,1.96,72.61,0.00,0.00,0.95,0.00,3
96002,0.58,5.95,5.97,84.07,0.00,0.00,3.50,0.00,3


In [31]:
# Soil class
df = pd.read_csv(f"{ATTRDIR}/camels-gb_attributes/CAMELS_GB_soil_attributes.csv", index_col='gauge_id')
df = df.drop(columns=['sand_perc_missing', 'silt_perc_missing','clay_perc_missing', 'organic_perc_missing','bulkdens_missing', 'tawc_missing',
                      'porosity_cosby_missing', 'conductivity_cosby_missing','conductivity_hypres_missing', 'root_depth_missing',
                      'soil_depth_pelletier_missing','porosity_hypres_missing','bulkdens_5','bulkdens_50','bulkdens_95', 'tawc_5', 'tawc_50','tawc_95',
                     'porosity_cosby_5','porosity_cosby_50','porosity_cosby_95','conductivity_cosby_5','conductivity_cosby_50','conductivity_cosby_95','porosity_hypres_5',
                      'porosity_hypres_50','porosity_hypres_95','conductivity_hypres_5','conductivity_hypres_50','conductivity_hypres_95','root_depth_5','root_depth_50',
                     'root_depth_95','soil_depth_pelletier_5','soil_depth_pelletier_50','soil_depth_pelletier_95'])

dataframes.append(df)

print(len(df.dropna()))
df

671


,sand_perc,silt_perc,clay_perc,organic_perc,bulkdens,tawc,porosity_cosby,porosity_hypres,conductivity_cosby,conductivity_hypres,root_depth,soil_depth_pelletier
gauge_id,,,,,,,,,,,,
10002,40.98,35.60,23.42,5.22,1.20,116.79,43.81,51.13,1.34,1.49,1.20,13.21
10003,43.79,37.96,18.25,1.58,1.29,88.47,43.61,47.54,1.58,1.50,1.01,7.52
1001,39.25,31.97,28.79,19.32,0.74,182.04,43.86,66.45,1.16,1.87,1.30,14.54
101002,70.99,15.36,13.66,0.43,1.55,82.46,39.91,38.36,3.26,1.51,1.26,2.80
101005,71.01,15.56,13.45,0.45,1.54,83.52,39.92,38.43,3.52,1.53,1.25,3.38
...,...,...,...,...,...,...,...,...,...,...,...,...
95001,47.75,24.31,27.93,12.64,1.02,121.02,42.69,56.49,1.14,1.33,0.91,1.03
96001,41.07,26.97,31.97,25.52,0.53,197.41,43.49,73.01,1.01,1.70,1.24,5.48
96002,42.42,24.10,33.48,12.65,0.96,128.77,43.24,58.78,0.90,1.04,1.06,1.76


## Add hydrological signatures

In [32]:
basin_ids = pd.read_csv(f"{ATTRDIR}/camels-gb_attributes/CAMELS_GB_topographic_attributes.csv").gauge_id.to_list()

In [33]:
# Hydrological Signatures - groundwater
df = pd.read_table(f"{ATTRDIR}/hydrological_signatures/McMillan_Groundwater_signatures.txt", sep=',',
                   usecols=['TotalRR','Recession_a_Seasonality','AverageStorage','BFI','BaseflowRecessionK'])
df['gauge_id'] = basin_ids
df = df.set_index('gauge_id')
dataframes.append(df)

print(len(df.dropna()))
df

666


,TotalRR,Recession_a_Seasonality,AverageStorage,BFI,BaseflowRecessionK
gauge_id,,,,,
10002,0.595548,1.095501,75.383129,0.629431,0.076470
10003,0.565804,1.189070,73.252520,0.733916,0.049073
1001,0.617964,1.614870,53.336130,0.383689,0.184039
101002,0.451053,1.264968,176.272796,0.660207,0.100224
101005,0.368960,1.619237,156.746503,0.543259,0.102930
...,...,...,...,...,...
95001,0.823405,0.553715,139.699794,0.715822,0.072537
96001,0.712254,1.485388,64.400818,0.294134,0.218483
96002,0.797903,0.801369,58.242655,0.421639,0.157645


In [34]:
# Hydrological Signatures - surface water
df = pd.read_table(f"{ATTRDIR}/hydrological_signatures/McMillan_Overland_signatures.txt", sep=',',
                  usecols=['IE_thresh','SE_thresh','Storage_thresh','SE_slope'])

df['gauge_id'] = basin_ids
df = df.set_index('gauge_id')
dataframes.append(df)

print(len(df.dropna()))

669


In [35]:
df.fillna(0).astype(bool).sum(axis=0)

IE_thresh         669
SE_thresh         669
SE_slope          669
Storage_thresh    669
dtype: int64

## Concat Dataframes and store output

In [36]:
# Create only CAMELS-GB attrs dataframe
df_out = pd.concat(dataframes, axis=1)
# Drop all rows with NaN values
df_out = df_out.dropna()
df_out.to_csv(f'{RESULTS}/rf_feature_selection/camels_attributes_hydrological_signatures_rf_new.csv')

In [37]:
df_out

,gauge_elev,area,dpsbar,elev_mean,p_mean,pet_mean,aridity,p_seasonality,frac_snow,high_prec_freq,...,soil_depth_pelletier,TotalRR,Recession_a_Seasonality,AverageStorage,BFI,BaseflowRecessionK,IE_thresh,SE_thresh,SE_slope,Storage_thresh
gauge_id,,,,,,,,,,,,,,,,,,,,,
10002,8.5,325.72,42.3,87.0,2.29,1.26,0.55,-0.03,0.03,13.96,...,13.21,0.595548,1.095501,75.383129,0.629431,0.076470,5.130614,9.646516,0.358693,19.864889
10003,3.8,532.29,55.1,108.0,2.31,1.25,0.54,-0.01,0.03,13.24,...,7.52,0.565804,1.189070,73.252520,0.733916,0.049073,6.962162,12.469081,0.297369,22.192499
1001,13.0,158.18,29.9,80.0,2.65,1.15,0.44,-0.15,0.03,10.98,...,14.54,0.617964,1.614870,53.336130,0.383689,0.184039,3.954470,11.335577,0.524633,20.787614
101002,10.4,30.61,80.0,60.0,2.31,1.45,0.63,-0.23,0.01,20.60,...,2.80,0.451053,1.264968,176.272796,0.660207,0.100224,8.861238,13.219895,0.238776,23.110227
101005,17.2,24.31,87.0,92.0,2.29,1.45,0.63,-0.23,0.01,20.47,...,3.38,0.368960,1.619237,156.746503,0.543259,0.102930,7.662708,15.414602,0.287711,24.991252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95001,60.3,138.47,190.3,292.0,6.20,1.05,0.17,-0.25,0.03,8.16,...,1.03,0.823405,0.553715,139.699794,0.715822,0.072537,14.166420,34.582125,0.341729,45.606937
96001,23.2,193.72,55.1,175.0,3.06,1.10,0.36,-0.19,0.04,9.73,...,5.48,0.712254,1.485388,64.400818,0.294134,0.218483,0.778313,6.290605,0.540612,16.663462
96002,5.1,474.01,111.7,224.0,3.81,1.08,0.28,-0.28,0.03,8.53,...,1.76,0.797903,0.801369,58.242655,0.421639,0.157645,5.906311,12.944906,0.552875,23.798810
